In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import random
from collections import Counter


In [23]:
data = pd.read_csv('a3_train_final.tsv',sep='\t', header=None, names=['label', 'text'])

In [24]:
# Function to process labels
def process_labels(label):
    label_list = label.split("/")
    label_list = [l for l in label_list if l != "-1"]  # Drop -1 labels
    if len(label_list) == 1:
        return label_list[0]  # Return the single label
    else:
        counter = Counter(label_list)
        max_vote = max(counter.values())
        majority_labels = [l for l, count in counter.items() if count == max_vote]
        if len(majority_labels) == 1:
            return random.choice(majority_labels)  # Randomly choose from majority labels
        else:
            return -1

In [25]:
# Apply label processing
data['final_label'] = data['label'].apply(process_labels)

In [26]:
df = data[data['final_label'] != -1]
df.drop(['label'], axis=1, inplace=True)

C:\Users\Alex\AppData\Local\Temp\ipykernel_640\2372967172.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['label'], axis=1, inplace=True)


In [27]:
# Drop duplicates based on the "text" column
final_df= df.drop_duplicates(subset=['text'])

# Create a DataFrame containing only the dropped rows
dropped_rows_df = df[~df.index.isin(final_df.index)]

# Print the DataFrame with dropped duplicates
print("DataFrame with dropped duplicates:")
print(final_df)

# Print the DataFrame containing dropped rows
print("\nDataFrame containing dropped rows:")
print(dropped_rows_df)

DataFrame with dropped duplicates:
                                                    text final_label
0       I'll only consume if I know what's inside it....           1
1       It is easier to fool a million people than it...           0
2       NATURAL IMMUNITY  protected us since evolutio...           0
3       NATURAL IMMUNITY  protected us since evolutio...           0
4       Proud to have resisted. Proud of my husband, ...           0
...                                                  ...         ...
50063  🤣 keep your 💩 I already know 3 people who have...           0
50064  🤣🤣🤣 "JUST BECAUSE IT'S SAFE, DOESN'T MEAN IT D...           0
50065  🤣🤣🤣 I took the Vaccine because of work. If I d...           0
50066  🤨there's people already having severe side eff...           0
50067  🥦I ❤my covid vaccines and I'm so excited for m...           1

[47370 rows x 2 columns]

DataFrame containing dropped rows:
                                                    text final_label
7655  

In [28]:
X = df['text']
y = df['final_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
# Pipeline for Logistic Regression with TfidfVectorizer
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

In [34]:
param_grid = {
    'tfidf__max_features': [1000, 2000, 3000],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__C': [0.1, 1.0, 10.0],
}

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)


Best hyperparameters: {'clf__C': 10.0, 'tfidf__max_features': 3000, 'tfidf__ngram_range': (1, 2)}


In [35]:

# Evaluate on test set
accuracy = grid_search.best_estimator_.score(X_test, y_test)
print("Accuracy on test set:", accuracy)

Accuracy on test set: 0.8162532981530343
